<a href="https://colab.research.google.com/github/dgg32/crop_area/blob/main/Google_Earth_Engine_USA_Crops_Area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run me first

First of all, run the following cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=xQ3M6w0ddbgZD0sevboVnWnaQ1KFs0Na-1Lm-HGDLhQ&tc=4O-PsyKagKw6A6zfDyfSBIA9Bgna6HlkirodTDb2T8w&cc=7imlC6OCJVYoslUS2F8YSMJz3-WxJINKAuKAwV72BOE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJqXLQCO0p_7hA_Ev9cZpoo1QkbNZ6ypRga53wG4OijGQTlbsYWfzbM

Successfully saved authorization token.


In [3]:
startDate = '2015-01-01'
endDate = '2016-01-01'

areaPerPixel = ee.Image.pixelArea()

dataset = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(startDate, endDate)).select("cropland").first()
provinces = ee.FeatureCollection("FAO/GAUL/2015/level2");

targets = ["Corn", "Cotton", "Rice"]
target_index = 0

state_list = ee.List(["Alabama", "Colorado"]);

def calculate_area (state):
    def generate_collection(feature):
      geometry = feature.geometry();
      
                  
      o_dataset = dataset.clip(geometry);


      image = dataset.eq(target_index).selfMask().multiply(areaPerPixel).divide(1e6).set('system:time_start', startDate);
      
      
      reduce = image.select('cropland').reduceRegion(
                                                reducer= ee.Reducer.sum(),
                                                geometry= geometry,
                                                scale= 30,
                                                bestEffort= True
                                            );
            
      return feature.set(target, reduce.get("cropland"))

    filterLevel1 = ee.Filter.eq('ADM1_NAME', state);
    o_state = provinces.filter(filterLevel1);

    corn_areas = o_state.map(generate_collection)

    results = corn_areas.reduceColumns(ee.Reducer.sum(), [target])


    return results.values(["sum"]).get(0)

for target in targets:
    #target = "Corn";
    target_index = dataset.get("cropland_class_names").getInfo().index(target);

    results = state_list.map(calculate_area).reduce(ee.Reducer.sum());

    print (target, results.getInfo())

Corn 5691.660624604763
Cotton 1416.399430588669


EEException: ignored